In [1]:
minimum_precision = 4
# THIS IS AN INCLUSIVE RANGE!!
maximum_precision = 16
bits_per_registry = 6
registry_per_word = (32 // bits_per_registry)

In [2]:
body = """
use crate::prelude::*;

#[inline]
pub fn count_{number_of_registers}(registers: &[u32; {number_of_words}]) -> (usize, f32) {{
{assign_words}

{assign_registers}

    (
{check_zeros},
{harmonic_sum}
    )
}}
"""

dispatcher = """
use crate::prelude::*;

#[inline]
pub fn dispatch_specialized_count<const N: usize>(
    registers: &[u32; N],
) -> (usize, f32) {{
    match N {{
{cases}
        _ => unimplemented!(),
    }}
}}

"""

In [3]:
import shutil
from math import ceil
import os

path = "src/specialized_counts"

if os.path.exists(path):
    shutil.rmtree(path)
    
os.makedirs(path)

module = []
cases = []

for precision in range(minimum_precision, maximum_precision - 3):
    number_of_registers = 1 << precision
    number_of_words = int(ceil(number_of_registers / registry_per_word))
    
    assign_words = []
    for word_number in range(number_of_words):
        assign_words.append(
            f"\tlet word_{word_number} = registers[{word_number}];"
        )
        
    assign_registers = []
    check_zeros = []
    harmonic_sum = []
    word_number = -1
    for register_number in range(number_of_registers):
        remainder = register_number % registry_per_word
        if remainder == 0:
            word_number += 1
            assign_registers.append(
                f"\tlet register_{register_number} = word_{word_number} & LOWER_REGISTER_MASK;"
            )
        elif remainder == 1:
            assign_registers.append(
                f"\tlet register_{register_number} = (word_{word_number} >> NUMBER_OF_BITS_PER_REGISTER) & LOWER_REGISTER_MASK;"
            )
        else:
            assign_registers.append(
                f"\tlet register_{register_number} = (word_{word_number} >> {remainder} * NUMBER_OF_BITS_PER_REGISTER) & LOWER_REGISTER_MASK;"
            )
        check_zeros.append(f"\t\t(register_{register_number} == 0) as usize")
        harmonic_sum.append(f"\t\t1.0_f32 / (1u64 << register_{register_number}) as f32")
        
    completed_body = body.format(
        number_of_registers=number_of_registers,
        number_of_words=number_of_words,
        assign_words="\n".join(assign_words),
        assign_registers="\n".join(assign_registers),
        check_zeros=" +\n".join(check_zeros),
        harmonic_sum=" +\n".join(harmonic_sum)
    )
    
    cases.append(
        f"\t\t{number_of_words} => count_{number_of_registers}(unsafe {{ core::mem::transmute(registers) }}),"
    )
    module.append(f"pub mod count_{number_of_registers};")
    module.append(f"pub use count_{number_of_registers}::*;")
    
    with open(f"{path}/count_{number_of_registers}.rs", "w") as f:
        f.write(completed_body)
        
module.append("pub mod specialized_counts_dispatcher;")
module.append("pub use specialized_counts_dispatcher::*;")

with open(f"{path}/specialized_counts_dispatcher.rs", "w") as f:
    f.write(dispatcher.format(cases="\n".join(cases)))

with open(f"{path}/mod.rs", "w") as f:
    f.write("\n".join(module))
    